#### Trabajo sobre la base de datos de la **"Encyclopaedia of exoplanetary systems"** de la Unión Europea

##### En lo que sigue se propone trabajar con bases de datos en observatorios virtuales: usando un catálogo de exoplanetas de la base de datos [exoplanets.eu](https://www.exoplanets.eu)

**Enlaces útiles:**
- [Encyclopaedia of exoplanetary systems](https://www.exoplanets.eu) - Base de datos principal
- [Catálogo CSV](https://www.exoplanets.eu/catalog/csv/) - Descarga de datos en formato CSV
- [Documentación de la API](https://www.exoplanets.eu/api/) - Para acceso programático

In [24]:
# Verificación de instalación de pyvo

import pyvo
print(f"PyVO version: {pyvo.__version__}")

PyVO version: 1.5.2


In [25]:
# Seleccionamos el servicio de exoplanetas
# Usamos la URL correcta del servicio TAP de Paris Observatory

service = pyvo.dal.TAPService("http://voparis-tap-planeto.obspm.fr/tap")

# Verificamos que el servicio esté funcionando
print("Conectando al servicio TAP...")
print(f"URL del servicio: {service.baseurl}")

# Listamos las tablas disponibles usando una consulta ADQL
try:
    print("Obteniendo lista de tablas...")
    
    # Consulta para obtener información de las tablas
    table_query = "SELECT table_name, table_type FROM TAP_SCHEMA.tables"
    table_results = service.search(table_query)
    
    print(f"Número de tablas disponibles: {len(table_results)}")
    print("\nPrimeras tablas encontradas:")
    
    # Convertir a lista para mejor manejo
    tables_list = []
    for row in table_results:
        tables_list.append({
            'name': str(row['table_name']), 
            'type': str(row['table_type'])
        })
    
    # Mostrar primeras 10 tablas
    for i, table in enumerate(tables_list[:10]):
        print(f"- {table['name']} ({table['type']})")
    
    print("\nBuscando tablas relacionadas con exoplanetas...")
    exo_tables = [table for table in tables_list if 'exoplanet' in table['name'].lower() or 'planet' in table['name'].lower()]
    
    if exo_tables:
        print("Tablas de exoplanetas encontradas:")
        for table in exo_tables:
            print(f"- {table['name']}")
    else:
        print("No se encontraron tablas específicas de exoplanetas en los nombres.")
        print("\nMostrando todas las tablas disponibles:")
        for table in tables_list:
            print(f"- {table['name']}")
        
except Exception as e:
    print(f"Error al obtener tablas: {e}")
    print("Intentaremos con una consulta más básica...")

Conectando al servicio TAP...
URL del servicio: http://voparis-tap-planeto.obspm.fr/tap
Obteniendo lista de tablas...
Número de tablas disponibles: 41

Primeras tablas encontradas:
- tap_schema.schemas (table)
- tap_schema.tables (table)
- tap_schema.columns (table)
- tap_schema.keys (table)
- tap_schema.key_columns (table)
- tap_schema.groups (table)
- iks.epn_core (table)
- basecom.epn_core (table)
- exoplanet.epn_core (table)
- vims_satellites.epn_core (table)

Buscando tablas relacionadas con exoplanetas...
Tablas de exoplanetas encontradas:
- exoplanet.epn_core
- planets.epn_core
- spectro_planets.epn_core
- hst_planeto.epn_core
Número de tablas disponibles: 41

Primeras tablas encontradas:
- tap_schema.schemas (table)
- tap_schema.tables (table)
- tap_schema.columns (table)
- tap_schema.keys (table)
- tap_schema.key_columns (table)
- tap_schema.groups (table)
- iks.epn_core (table)
- basecom.epn_core (table)
- exoplanet.epn_core (table)
- vims_satellites.epn_core (table)

Buscand

In [26]:
# Realizamos un query para obtener información sobre exoplanetas
# Usamos la tabla exoplanet.epn_core que encontramos

print("Preparando consulta para la tabla de exoplanetas...")

# Primero, veamos qué columnas tiene la tabla exoplanet.epn_core
try:
    columns_query = "SELECT column_name, datatype FROM TAP_SCHEMA.columns WHERE table_name = 'exoplanet.epn_core'"
    columns_result = service.search(columns_query)
    
    print("Columnas disponibles en exoplanet.epn_core:")
    columns_list = []
    for row in columns_result:
        column_info = f"{row['column_name']} ({row['datatype']})"
        columns_list.append(str(row['column_name']))
        print(f"- {column_info}")
    
    print(f"\nTotal de columnas: {len(columns_list)}")
    
    # Crear una consulta usando algunas columnas principales
    if 'target_name' in columns_list:
        query = "SELECT TOP 10 target_name, target_class FROM exoplanet.epn_core"
    else:
        # Consulta más genérica
        query = "SELECT TOP 10 * FROM exoplanet.epn_core"
    
    print(f"Query a ejecutar: {query}")
    
except Exception as e:
    print(f"Error al obtener columnas: {e}")
    # Consulta de fallback
    query = "SELECT TOP 5 * FROM exoplanet.epn_core"
    print(f"Usando consulta de fallback: {query}")

Preparando consulta para la tabla de exoplanetas...
Columnas disponibles en exoplanet.epn_core:
- granule_uid (char)
- granule_gid (char)
- obs_id (char)
- dataproduct_type (char)
- target_name (char)
- target_class (char)
- time_min (double)
- time_max (double)
- time_sampling_step_min (double)
- time_sampling_step_max (double)
- time_exp_min (double)
- time_exp_max (double)
- spectral_range_min (double)
- spectral_range_max (double)
- spectral_sampling_step_min (double)
- spectral_sampling_step_max (double)
- spectral_resolution_min (double)
- spectral_resolution_max (double)
- c1min (double)
- c1max (double)
- c2min (double)
- c2max (double)
- c3min (double)
- c3max (double)
- s_region (char)
- c1_resol_min (double)
- c1_resol_max (double)
- c2_resol_min (double)
- c2_resol_max (double)
- c3_resol_min (double)
- c3_resol_max (double)
- spatial_frame_type (char)
- incidence_min (double)
- incidence_max (double)
- emergence_min (double)
- emergence_max (double)
- phase_min (double)
- 

In [27]:
# Ejecutamos el query y mostramos los resultados
try:
    print("Ejecutando consulta...")
    results = service.search(query)
    
    print(f"Consulta exitosa. Número de filas encontradas: {len(results)}")
    
    if len(results) > 0:
        print("\nPrimeros resultados:")
        print("-" * 50)
        
        # Mostrar información de las columnas
        if hasattr(results, 'fieldnames'):
            print(f"Columnas disponibles: {results.fieldnames}")
            print()
        
        # Mostrar las primeras filas (manejo correcto)
        count = 0
        for row in results:
            if count >= 5:  # Limitar a 5 filas
                break
            print(f"Fila {count+1}:")
            # Acceder a los datos por nombre de columna
            if 'target_name' in results.fieldnames:
                print(f"  Nombre: {row['target_name']}")
                print(f"  Clase: {row['target_class']}")
            else:
                # Si no podemos acceder por nombre, mostrar toda la fila
                print(f"  {row}")
            print()
            count += 1
    else:
        print("No se encontraron resultados para la consulta.")
        
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")
    print("Intentemos una consulta más básica...")
    
    # Consulta alternativa para verificar la conectividad
    try:
        basic_query = "SELECT TOP 3 table_name FROM TAP_SCHEMA.tables"
        basic_results = service.search(basic_query)
        print(f"\nConsulta básica exitosa. Tablas encontradas:")
        for row in basic_results:
            print(f"- {row['table_name']}")
    except Exception as e2:
        print(f"Error en consulta básica: {e2}")

Ejecutando consulta...
Consulta exitosa. Número de filas encontradas: 10

Primeros resultados:
--------------------------------------------------
Columnas disponibles: ('target_name', 'target_class')

Fila 1:
  Nombre: 109 Psc b
  Clase: exoplanet

Fila 2:
  Nombre: 112 Psc b
  Clase: exoplanet

Fila 3:
  Nombre: 112 Psc c
  Clase: exoplanet

Fila 4:
  Nombre: 11 Com Ab
  Clase: exoplanet

Fila 5:
  Nombre: 11 UMi b
  Clase: exoplanet

Consulta exitosa. Número de filas encontradas: 10

Primeros resultados:
--------------------------------------------------
Columnas disponibles: ('target_name', 'target_class')

Fila 1:
  Nombre: 109 Psc b
  Clase: exoplanet

Fila 2:
  Nombre: 112 Psc b
  Clase: exoplanet

Fila 3:
  Nombre: 112 Psc c
  Clase: exoplanet

Fila 4:
  Nombre: 11 Com Ab
  Clase: exoplanet

Fila 5:
  Nombre: 11 UMi b
  Clase: exoplanet



In [31]:
# Consultas más avanzadas sobre exoplanetas
print("=== ANÁLISIS AVANZADO DE EXOPLANETAS ===\n")

# 1. Obtener exoplanetas con información orbital completa
print("1. Exoplanetas con datos orbitales completos:")
orbital_query = """
SELECT TOP 15 
    target_name, 
    star_name,
    mass, 
    radius, 
    period, 
    semi_major_axis,
    star_distance,
    discovered
FROM exoplanet.epn_core 
WHERE mass IS NOT NULL 
    AND radius IS NOT NULL 
    AND period IS NOT NULL 
    AND semi_major_axis IS NOT NULL
ORDER BY discovered DESC
"""

try:
    orbital_results = service.search(orbital_query)
    print(f"Encontrados {len(orbital_results)} exoplanetas con datos completos\n")
    
    for i, planet in enumerate(orbital_results):
        if i >= 5:  # Mostrar solo los primeros 5
            break
        print(f"{i+1}. {planet['target_name']}")
        print(f"   Estrella: {planet['star_name']}")
        print(f"   Masa: {planet['mass']:.3f} M⊕")
        print(f"   Radio: {planet['radius']:.3f} R⊕")
        print(f"   Período: {planet['period']:.2f} días")
        print(f"   Semieje mayor: {planet['semi_major_axis']:.3f} AU")
        print(f"   Distancia estelar: {planet['star_distance']:.1f} pc")
        print(f"   Descubierto: {planet['discovered']}")
        print()
        
except Exception as e:
    print(f"Error en consulta orbital: {e}")

# 2. Estadísticas básicas
print("\n2. Estadísticas de la base de datos:")
stats_query = "SELECT COUNT(*) as total_exoplanets FROM exoplanet.epn_core"

try:
    stats_results = service.search(stats_query)
    total = stats_results[0]['total_exoplanets']
    print(f"Total de exoplanetas en la base de datos: {total}")
    
    # Contar por clase
    class_query = "SELECT target_class, COUNT(*) as count FROM exoplanet.epn_core GROUP BY target_class"
    class_results = service.search(class_query)
    
    print("\nDistribución por clase:")
    for row in class_results:
        print(f"- {row['target_class']}: {row['count']} objetos")
        
except Exception as e:
    print(f"Error en estadísticas: {e}")

print("\n✓ Análisis completado!")

=== ANÁLISIS AVANZADO DE EXOPLANETAS ===

1. Exoplanetas con datos orbitales completos:
Encontrados 15 exoplanetas con datos completos

1. TOI-1743 b
   Estrella: TOI-1743
   Masa: 0.015 M⊕
   Radio: 0.163 R⊕
   Período: 4.27 días
   Semieje mayor: 0.036 AU
   Distancia estelar: 41.3 pc
   Descubierto: 2025

2. TOI-2031 Ab
   Estrella: TOI-2031 A
   Masa: 0.800 M⊕
   Radio: 1.267 R⊕
   Período: 5.72 días
   Semieje mayor: 0.066 AU
   Distancia estelar: 276.4 pc
   Descubierto: 2025

3. TOI-1203 Ab
   Estrella: TOI-1203 A
   Masa: 0.011 M⊕
   Radio: 0.136 R⊕
   Período: 4.16 días
   Semieje mayor: 0.049 AU
   Distancia estelar: 65.0 pc
   Descubierto: 2025

4. TOI-1203 Ad
   Estrella: TOI-1203 A
   Masa: 0.023 M⊕
   Radio: 0.260 R⊕
   Período: 25.50 días
   Semieje mayor: 0.163 AU
   Distancia estelar: 65.0 pc
   Descubierto: 2025

5. TOI-1846 b
   Estrella: TOI-1846
   Masa: 0.014 M⊕
   Radio: 0.160 R⊕
   Período: 3.93 días
   Semieje mayor: 0.036 AU
   Distancia estelar: 47.2 pc
   De

In [ ]:
# Quiero buscar un planeta particular, genero una función que busque por nombre, que seria algo así:

# el input df es el DataFrame que contiene los datos de exoplanetas

def buscar_planeta(df, target_name):
    """
    Busca un exoplaneta por su nombre en el DataFrame
    """
    resultado = df[df['name'].str.contains(target_name, case=False, na=False)]
    return resultado

In [48]:
# Crear DataFrame con todos los datos de exoplanetas
import pandas as pd

print("Obteniendo todos los datos de exoplanetas para crear DataFrame...")

try:
    # Consulta para obtener todos los exoplanetas con las columnas principales
    full_query = """
    SELECT 
        target_name as name,
        star_name,
        mass, 
        radius, 
        period, 
        semi_major_axis,
        eccentricity,
        inclination,
        star_distance,
        star_spec_type,
        star_mass,
        star_radius,
        star_teff,
        discovered,
        detection_type,
        ra,
        dec
    FROM exoplanet.epn_core 
    WHERE target_name IS NOT NULL
    """
    
    # Ejecutar consulta
    full_results = service.search(full_query)
    print(f"Obtenidos {len(full_results)} exoplanetas")
    
    # Convertir a DataFrame de pandas
    data = []
    for row in full_results:
        data.append({
            'name': row['name'],
            'star_name': row['star_name'],
            'mass': row['mass'],
            'radius': row['radius'],
            'period': row['period'],
            'semi_major_axis': row['semi_major_axis'],
            'eccentricity': row['eccentricity'],
            'inclination': row['inclination'],
            'star_distance': row['star_distance'],
            'star_spec_type': row['star_spec_type'],
            'star_mass': row['star_mass'],
            'star_radius': row['star_radius'],
            'star_teff': row['star_teff'],
            'discovered': row['discovered'],
            'detection_type': row['detection_type'],
            'ra': row['ra'],
            'dec': row['dec']
        })
    
    df = pd.DataFrame(data)
    
    print(f"✓ DataFrame creado con {len(df)} exoplanetas")
    print(f"Columnas disponibles: {list(df.columns)}")
    print(f"\nPrimeras filas:")
    print(df.head())
    
except Exception as e:
    print(f"Error al crear DataFrame: {e}")
    # Crear DataFrame vacío como fallback
    df = pd.DataFrame()

Obteniendo todos los datos de exoplanetas para crear DataFrame...
Obtenidos 7762 exoplanetas
✓ DataFrame creado con 7762 exoplanetas
Columnas disponibles: ['name', 'star_name', 'mass', 'radius', 'period', 'semi_major_axis', 'eccentricity', 'inclination', 'star_distance', 'star_spec_type', 'star_mass', 'star_radius', 'star_teff', 'discovered', 'detection_type', 'ra', 'dec']

Primeras filas:
        name star_name   mass  radius    period  semi_major_axis  \
0  109 Psc b   109 Psc  5.743   1.152   1075.40            2.051   
1  112 Psc b   112 Psc    NaN     NaN      4.40            0.054   
2  112 Psc c   112 Psc  9.866     NaN  36336.70           22.210   
3  11 Com Ab  11 Com A    NaN     NaN    326.03            1.290   
4   11 UMi b    11 UMi    NaN     NaN    516.22            1.540   

   eccentricity  inclination  star_distance star_spec_type  star_mass  \
0         0.104       86.116        32.5600          G5 IV       1.13   
1         0.376          NaN        31.7627         

In [46]:
nuoct = buscar_planeta(df, "nu Octantis") 

print(nuoct)

Empty DataFrame
Columns: [name, star_name, mass, radius, period, semi_major_axis, eccentricity, inclination, star_distance, star_spec_type, star_mass, star_radius, star_teff, discovered, detection_type, ra, dec]
Index: []


In [43]:
# SOLUCIÓN FINAL: Buscar nu Oct b de forma directa

print("=== BÚSQUEDA FINAL DE NU OCT B ===")

# Verificar que el DataFrame existe y tiene datos
print(f"DataFrame shape: {df.shape}")
print(f"Columnas: {list(df.columns)}")

# Buscar directamente "nu Oct" en el nombre del planeta
result_planet = df[df['name'].str.contains("nu Oct", case=False, na=False)]
print(f"\nBúsqueda en nombre del planeta: {len(result_planet)} resultados")

# Buscar "nu Oct" en el nombre de la estrella  
result_star = df[df['star_name'].str.contains("nu Oct", case=False, na=False)]
print(f"Búsqueda en nombre de la estrella: {len(result_star)} resultados")

# Combinar resultados
all_results = pd.concat([result_planet, result_star]).drop_duplicates()
print(f"Total de resultados únicos: {len(all_results)}")

if len(all_results) > 0:
    print("\n=== INFORMACIÓN DE NU OCT B ===")
    for idx, planet in all_results.iterrows():
        print(f"Planeta: {planet['name']}")
        print(f"Estrella: {planet['star_name']}")
        print(f"Masa: {planet['mass']} M⊕")
        print(f"Período: {planet['period']} días")
        print(f"Distancia: {planet['star_distance']} pc")
        print(f"Descubierto: {planet['discovered']}")
        print("-" * 40)
        
    # Asignar resultado a la variable original
    nuoct = all_results
    print(f"\n✓ Variable 'nuoct' creada con {len(nuoct)} registro(s)")
else:
    print("\n⚠️ No se encontraron resultados para 'nu Oct'")
    
    # Mostrar algunos nombres de ejemplo para verificar el formato
    print("\nEjemplos de nombres en la base de datos:")
    sample_names = df['name'].dropna().head(10)
    for name in sample_names:
        print(f"- {name}")
    
    print("\nEjemplos de nombres de estrellas:")
    sample_stars = df['star_name'].dropna().head(10)
    for star in sample_stars:
        print(f"- {star}")

=== BÚSQUEDA FINAL DE NU OCT B ===
DataFrame shape: (7762, 17)
Columnas: ['name', 'star_name', 'mass', 'radius', 'period', 'semi_major_axis', 'eccentricity', 'inclination', 'star_distance', 'star_spec_type', 'star_mass', 'star_radius', 'star_teff', 'discovered', 'detection_type', 'ra', 'dec']

Búsqueda en nombre del planeta: 1 resultados
Búsqueda en nombre de la estrella: 1 resultados
Total de resultados únicos: 1

=== INFORMACIÓN DE NU OCT B ===
Planeta: nu Oct b
Estrella: nu Oct
Masa: 2.13 M⊕
Período: 409.3 días
Distancia: 22.1 pc
Descubierto: 2009
----------------------------------------

✓ Variable 'nuoct' creada con 1 registro(s)


In [45]:
# SOLUCIÓN ROBUSTA Y COMPLETA PARA NU OCT B

print("🔍 BÚSQUEDA COMPLETA DE NU OCT B")
print("=" * 50)

# 1. Verificar que el DataFrame existe y es válido
if 'df' in locals() and not df.empty:
    print(f"✓ DataFrame válido con {len(df)} exoplanetas")
    
    # 2. Buscar el planeta de diferentes maneras
    methods = [
        ("nu Oct", "Búsqueda exacta 'nu Oct'"),
        ("nu Octantis", "Búsqueda 'nu Octantis'"), 
        ("Oct", "Búsqueda parcial 'Oct'"),
        ("Octant", "Búsqueda parcial 'Octant'")
    ]
    
    found = False
    for search_term, description in methods:
        print(f"\n{description}:")
        
        # Buscar en nombre del planeta
        planet_matches = df[df['name'].str.contains(search_term, case=False, na=False)]
        
        # Buscar en nombre de la estrella
        star_matches = df[df['star_name'].str.contains(search_term, case=False, na=False)]
        
        # Combinar resultados
        all_matches = pd.concat([planet_matches, star_matches]).drop_duplicates()
        
        if len(all_matches) > 0:
            found = True
            print(f"  ✓ Encontrados {len(all_matches)} resultado(s)")
            
            for idx, row in all_matches.iterrows():
                print(f"    - Planeta: {row['name']}")
                print(f"      Estrella: {row['star_name']}")
                print(f"      Masa: {row['mass']} M⊕")
                print(f"      Período: {row['period']} días")
                
                # Si es nu Oct b, guardar como resultado principal
                if 'nu Oct' in str(row['name']).lower():
                    nuoct = all_matches
                    print(f"    → Guardado como 'nuoct'")
            
            if 'nu Oct' in search_term:
                break  # Si encontramos con nu Oct, no necesitamos seguir
        else:
            print(f"  ✗ No se encontraron resultados")
    
    # 3. Resultado final
    if found and 'nuoct' in locals():
        print(f"\n🎯 RESULTADO FINAL:")
        print(f"Variable 'nuoct' creada exitosamente")
        print(f"Información del exoplaneta nu Oct b:")
        print(nuoct[['name', 'star_name', 'mass', 'period', 'star_distance', 'discovered']].to_string())
    elif not found:
        print(f"\n⚠️ No se encontró 'nu Oct' en la base de datos")
        print("Mostrando algunos ejemplos de nombres para referencia:")
        sample_names = df['name'].dropna().sample(10) if len(df) > 10 else df['name'].dropna()
        for name in sample_names:
            if 'oct' in name.lower():
                print(f"  - {name} ⭐")
            else:
                print(f"  - {name}")
                
else:
    print("❌ Error: DataFrame 'df' no está disponible")
    print("Ejecuta primero la celda que crea el DataFrame desde el servicio TAP")

🔍 BÚSQUEDA COMPLETA DE NU OCT B
✓ DataFrame válido con 7762 exoplanetas

Búsqueda exacta 'nu Oct':
  ✓ Encontrados 1 resultado(s)
    - Planeta: nu Oct b
      Estrella: nu Oct
      Masa: 2.13 M⊕
      Período: 409.3 días

🎯 RESULTADO FINAL:
Variable 'nuoct' creada exitosamente
Información del exoplaneta nu Oct b:
Empty DataFrame
Columns: [name, star_name, mass, period, star_distance, discovered]
Index: []


In [39]:
# Mejorar la función de búsqueda y explorar los datos
def buscar_planeta_mejorado(df, search_term):
    """
    Busca un exoplaneta por nombre (planeta o estrella) de forma más flexible
    """
    print(f"Buscando: '{search_term}'")
    
    # Buscar en nombre del planeta
    resultado_planeta = df[df['name'].str.contains(search_term, case=False, na=False)]
    
    # Buscar en nombre de la estrella
    resultado_estrella = df[df['star_name'].str.contains(search_term, case=False, na=False)]
    
    # Combinar resultados
    resultado = pd.concat([resultado_planeta, resultado_estrella]).drop_duplicates()
    
    print(f"Encontrados {len(resultado)} resultados")
    return resultado

# Buscar "nu Octantis" o variaciones
print("=== BÚSQUEDA DE NU OCTANTIS ===")
result1 = buscar_planeta_mejorado(df, "nu Octantis")
if len(result1) == 0:
    print("No encontrado. Probando variaciones...")
    result2 = buscar_planeta_mejorado(df, "Octantis")
    if len(result2) == 0:
        result3 = buscar_planeta_mejorado(df, "nu Oct")
        if len(result3) == 0:
            print("No se encontró. Exploremos qué nombres contienen 'Oct':")
            oct_results = buscar_planeta_mejorado(df, "Oct")
            if len(oct_results) > 0:
                print(oct_results[['name', 'star_name']].head(10))
        else:
            print("Resultados con 'nu Oct':")
            print(result3[['name', 'star_name', 'mass', 'period']])
    else:
        print("Resultados con 'Octantis':")
        print(result2[['name', 'star_name', 'mass', 'period']])
else:
    print("Resultados encontrados:")
    print(result1[['name', 'star_name', 'mass', 'period']])

=== BÚSQUEDA DE NU OCTANTIS ===
Buscando: 'nu Octantis'
Encontrados 0 resultados
No encontrado. Probando variaciones...
Buscando: 'Octantis'
Encontrados 0 resultados
Buscando: 'nu Oct'
Encontrados 1 resultados
Resultados con 'nu Oct':
          name star_name  mass  period
6017  nu Oct b    nu Oct  2.13   409.3


In [40]:
# Información completa de nu Oct b
print("=== INFORMACIÓN COMPLETA DE NU OCT B ===")

nuoct = buscar_planeta_mejorado(df, "nu Oct")
if len(nuoct) > 0:
    planeta = nuoct.iloc[0]  # Tomar el primer resultado
    
    print(f"Nombre del planeta: {planeta['name']}")
    print(f"Estrella anfitriona: {planeta['star_name']}")
    print()
    
    print("PROPIEDADES DEL PLANETA:")
    print(f"  Masa: {planeta['mass']:.2f} M⊕" if pd.notna(planeta['mass']) else "  Masa: No disponible")
    print(f"  Radio: {planeta['radius']:.2f} R⊕" if pd.notna(planeta['radius']) else "  Radio: No disponible")
    print(f"  Período orbital: {planeta['period']:.1f} días" if pd.notna(planeta['period']) else "  Período: No disponible")
    print(f"  Semieje mayor: {planeta['semi_major_axis']:.3f} AU" if pd.notna(planeta['semi_major_axis']) else "  Semieje mayor: No disponible")
    print(f"  Excentricidad: {planeta['eccentricity']:.3f}" if pd.notna(planeta['eccentricity']) else "  Excentricidad: No disponible")
    print(f"  Inclinación: {planeta['inclination']:.1f}°" if pd.notna(planeta['inclination']) else "  Inclinación: No disponible")
    print()
    
    print("PROPIEDADES DE LA ESTRELLA:")
    print(f"  Distancia: {planeta['star_distance']:.1f} pc" if pd.notna(planeta['star_distance']) else "  Distancia: No disponible")
    print(f"  Tipo espectral: {planeta['star_spec_type']}" if pd.notna(planeta['star_spec_type']) else "  Tipo espectral: No disponible")
    print(f"  Masa estelar: {planeta['star_mass']:.2f} M☉" if pd.notna(planeta['star_mass']) else "  Masa estelar: No disponible")
    print(f"  Radio estelar: {planeta['star_radius']:.2f} R☉" if pd.notna(planeta['star_radius']) else "  Radio estelar: No disponible")
    print(f"  Temperatura efectiva: {planeta['star_teff']:.0f} K" if pd.notna(planeta['star_teff']) else "  Temperatura efectiva: No disponible")
    print()
    
    print("INFORMACIÓN DE DESCUBRIMIENTO:")
    print(f"  Año de descubrimiento: {planeta['discovered']}" if pd.notna(planeta['discovered']) else "  Año: No disponible")
    print(f"  Método de detección: {planeta['detection_type']}" if pd.notna(planeta['detection_type']) else "  Método: No disponible")
    print()
    
    print("COORDENADAS:")
    print(f"  Ascensión recta: {planeta['ra']:.6f}°" if pd.notna(planeta['ra']) else "  RA: No disponible")
    print(f"  Declinación: {planeta['dec']:.6f}°" if pd.notna(planeta['dec']) else "  Dec: No disponible")
    
    # Mostrar todos los datos en formato DataFrame para referencia
    print("\n=== DATOS COMPLETOS (DataFrame) ===")
    print(nuoct.T)  # Transpuesto para mejor visualización
    
else:
    print("No se encontró el planeta nu Oct b")

=== INFORMACIÓN COMPLETA DE NU OCT B ===
Buscando: 'nu Oct'
Encontrados 1 resultados
Nombre del planeta: nu Oct b
Estrella anfitriona: nu Oct

PROPIEDADES DEL PLANETA:
  Masa: 2.13 M⊕
  Radio: No disponible
  Período orbital: 409.3 días
  Semieje mayor: 1.250 AU
  Excentricidad: 0.118
  Inclinación: 110.9°

PROPIEDADES DE LA ESTRELLA:
  Distancia: 22.1 pc
  Tipo espectral: K1III
  Masa estelar: 1.61 M☉
  Radio estelar: 5.81 R☉
  Temperatura efectiva: 4860 K

INFORMACIÓN DE DESCUBRIMIENTO:
  Año de descubrimiento: 2009
  Método de detección: Other#Radial Velocity

COORDENADAS:
  Ascensión recta: 325.370833°
  Declinación: -77.390000°

=== DATOS COMPLETOS (DataFrame) ===
                                  6017
name                          nu Oct b
star_name                       nu Oct
mass                              2.13
radius                             NaN
period                           409.3
semi_major_axis                   1.25
eccentricity                     0.118
inclinatio